In [2]:
import itertools
import pulp

def find_multiset(n, v_dict, m):
    """
    Solve for a multiset M in F2^n satisfying divisibility properties:
      - For each beta in F2^n \ {0}, sum_x (-1)^{<beta,x>} * mu[x] = 2^{v_beta+1} * d_beta
      - sum_x mu[x] = 2^m
    Additionally, enforce each d_beta to be an odd integer.

    Args:
        n (int): dimension of the F2 vector space
        v_dict (dict[int, int]): map from beta (encoded as int 1..2^n-1) to v_beta
        m (int): exponent for total multiset size (|M| = 2^m)

    Returns:
        (dict[int,int], dict[int,int]): solutions for mu(x) and d_beta
    """
    # Create MILP problem: minimize max multiplicity for uniformity
    prob = pulp.LpProblem("Multiset_Divisibility_OddDBeta", pulp.LpMinimize)

    # Variables: mu[x] >= 0 integer multiplicities
    mu = {x: pulp.LpVariable(f"mu_{x}", lowBound=0, cat='Integer')
          for x in range(2**n)}
    # d_beta integer (no bounds)
    d = {beta: pulp.LpVariable(f"d_{beta}", lowBound=None, cat='Integer')
         for beta in range(1, 2**n)}

    # Constraint: total size = 2^m
    prob += sum(mu[x] for x in mu) == 2**m, "TotalSize"

#     # Constraints per beta != 0
#     for beta in range(1, 2**n):
#         v = v_dict.get(beta, 0)
#         # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
#         expr = pulp.lpSum(((-1)**(bin(x & beta).count('1') & 1)) * mu[x]
#                           for x in mu)
#         prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"

    # Constraints per beta != 0
    for beta in v_dict.keys():
        v = v_dict.get(beta, 0)
        # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
        expr = pulp.lpSum(((-1)**(bin(x & beta).count('1') & 1)) * mu[x]
                          for x in mu)
        prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"
    
     # Auxiliary for uniformity: minimize the maximum mu[x]
    T = pulp.LpVariable("T", lowBound=0, cat='Integer')
    for x in mu:
        prob += mu[x] <= T
    prob += T, "Minimize_max_mu"

#     # Optional objective: find any feasible solution
#     prob += 0, "Objective"

    # Solve with GLPK (or swap to CBC with path if desired)
    result = prob.solve(pulp.GLPK_CMD(msg=True))
    if pulp.LpStatus[result] != 'Optimal':
        raise ValueError(f"No optimal solution found: status {pulp.LpStatus[result]}")

    # Extract solutions
    mu_sol = {x: int(mu[x].value()) for x in mu}
    d_sol = {beta: int(d[beta].value()) for beta in v_dict.keys()}
    return mu_sol, d_sol


if __name__ == "__main__":
    # Example usage for n=4
    n = 4
    # Specify some v_beta values
    v_dict = {1: 1}
    m = 4  # total size = 2^4 = 16

    mu_solution, d_solution = find_multiset(n, v_dict, m)
    print("Mu(x) values:", mu_solution)
    print("Odd d_beta values:", d_solution)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/145f368f968c499d8fad2d5a28da31df-pulp.lp -o /tmp/145f368f968c499d8fad2d5a28da31df-pulp.sol
Reading problem data from '/tmp/145f368f968c499d8fad2d5a28da31df-pulp.lp'...
18 rows, 18 columns, 65 non-zeros
18 integer variables, none of which are binary
63 lines were read
GLPK Integer Optimizer 5.0
18 rows, 18 columns, 65 non-zeros
18 integer variables, none of which are binary
Preprocessing...
18 rows, 18 columns, 65 non-zeros
18 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  8.000e+00  ratio =  8.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 18
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
18 rows, 18 columns, 65 non-zeros
      0: obj =   0.000000000e+00 inf =   1.600e+01 (1)
      1: obj =   1.600000000e+01 inf =   0.000e+00 (0)
*    16: obj =   1.000000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP

In [ ]:
import random, pulp

def build_base(n, v_dict, m):
    prob = pulp.LpProblem("Base", pulp.LpMinimize)
    mu = {x: pulp.LpVariable(f"mu_{x}", lowBound=0, cat='Integer')
          for x in range(2**n)}
    d  = {β: pulp.LpVariable(f"d_{β}", lowBound=None, cat='Integer')
          for β in range(2**n)}
    T  = pulp.LpVariable("T", lowBound=0, cat='Integer')

    # total‐size
    prob += sum(mu.values()) == 2**m
#     # divisibility + odd‐d
#     for β, v in v_dict.items():
#         expr = pulp.lpSum(((-1)**(bin(x&β).count('1')%2))*mu[x]
#                           for x in mu)
#         # force d[β] odd by writing 2*d[β]+1
#         prob += expr == 2**(v+1)*(2*d[β] + 1)
        
        # divisibility + odd‐d
    for β in range(1, 2**n):
        v = v_dict.get(β, 0)
        expr = pulp.lpSum(((-1)**(bin(x&β).count('1')%2))*mu[x]
                          for x in mu)
        # force d[β] odd by writing 2*d[β]+1
        prob += expr == 2**(v+1)*(2*d[β] + 1)
        
        
    # uniformity cap
    for x in mu:
        prob += mu[x] <= T

    return prob, mu, d, T

def random_objective_enumerate(n, v_dict, m, K=5):
    sols = []
    for i in range(K):
        prob, mu, d, T = build_base(n, v_dict, m)
        # *maximize* a random linear form on mu
        prob.sense = pulp.LpMaximize
        weights = {x: random.random() for x in mu}
        prob.setObjective(pulp.lpSum(weights[x]*mu[x] for x in mu))

        status = prob.solve(pulp.GLPK_CMD(msg=False))
        if pulp.LpStatus[status] != "Optimal":
            break

        mu_sol = {x:int(mu[x].value()) for x in mu}
        d_sol  = {β:int(d[β].value())   for β in d}
        sols.append((mu_sol, d_sol))
    return sols

# --- example ---
if __name__=="__main__":
    n, m = 4, 5
    v_dict = {1:2, 2:1, 12:2}
    solutions = random_objective_enumerate(n, v_dict, m, K=5)
    for i,(μ,d) in enumerate(solutions,1):
        print(f"Solution #{i}")
        print(" mu:",μ)
        print(" d:",d)
        print()
